In [1]:
import numpy as np

# -------------------------------
# HMM Parameters
# -------------------------------
states = ["Quiet", "Busy"]
observations = ["Low", "Medium", "High"]

pi = np.array([0.7, 0.3])  # initial state probabilities
A = np.array([[0.2, 0.8],  # transition matrix
              [0.7, 0.3]])
B = np.array([[0.6, 0.3, 0.1],  # emission probabilities
              [0.2, 0.3, 0.5]])

T = 5  # length of simulated sequence

# -------------------------------
# 1. Simulate states and observations
# -------------------------------
state_seq = np.zeros(T, dtype=int)
obs_seq = np.zeros(T, dtype=int)

# Initial state
state_seq[0] = np.random.choice(len(states), p=pi)
obs_seq[0] = np.random.choice(len(observations), p=B[state_seq[0]])

# Subsequent states and observations
for t in range(1, T):
    state_seq[t] = np.random.choice(len(states), p=A[state_seq[t-1]])
    obs_seq[t] = np.random.choice(len(observations), p=B[state_seq[t]])

print("Simulated hidden states:", [states[s] for s in state_seq])
print("Simulated observations:", [observations[o] for o in obs_seq])

# -------------------------------
# 2. Forward Algorithm
# -------------------------------
def forward(obs_seq, A, B, pi):
    N = A.shape[0]
    T = len(obs_seq)
    alpha = np.zeros((T, N))
    
    # Initialization
    alpha[0, :] = pi * B[:, obs_seq[0]]
    
    # Recursion
    for t in range(1, T):
        for j in range(N):
            alpha[t, j] = B[j, obs_seq[t]] * np.sum(alpha[t-1, :] * A[:, j])
    
    # Termination
    return np.sum(alpha[-1, :])

P_forward = forward(obs_seq, A, B, pi)
print("Forward probability P(O|lambda) =", P_forward)

# -------------------------------
# 3. Viterbi Algorithm
# -------------------------------
def viterbi(obs_seq, A, B, pi, state_names):
    N = A.shape[0]
    T = len(obs_seq)
    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)
    
    # Initialization
    delta[0, :] = pi * B[:, obs_seq[0]]
    
    # Recursion
    for t in range(1, T):
        for j in range(N):
            seq_probs = delta[t-1, :] * A[:, j]
            psi[t, j] = np.argmax(seq_probs)
            delta[t, j] = np.max(seq_probs) * B[j, obs_seq[t]]
    
    # Backtracking
    path = np.zeros(T, dtype=int)
    path[T-1] = np.argmax(delta[T-1, :])
    for t in range(T-2, -1, -1):
        path[t] = psi[t+1, path[t+1]]
    
    decoded_states = [state_names[s] for s in path]
    P_viterbi = delta[T-1, path[T-1]]
    
    return decoded_states, P_viterbi

decoded_states, P_viterbi = viterbi(obs_seq, A, B, pi, states)
print("Viterbi decoded hidden states:", decoded_states)
print("Viterbi path probability:", P_viterbi)


Simulated hidden states: ['Quiet', 'Quiet', 'Busy', 'Quiet', 'Busy']
Simulated observations: ['Low', 'Medium', 'High', 'Low', 'High']
Forward probability P(O|lambda) = 0.006588144000000001
Viterbi decoded hidden states: ['Quiet', 'Busy', 'Busy', 'Quiet', 'Busy']
Viterbi path probability: 0.00254016
